<a href="https://colab.research.google.com/github/KhanilReddy/ML1/blob/main/gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -U -q 'google-genai'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 1.8 MB/s eta 0:00:00


In [1]:
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

In [62]:
from google.colab import userdata
from google import genai
from google.genai import types
from pydantic import BaseModel
import json
import time

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

system_instructions = """
You are an AI model designed to classify IT support tickets.
Classify each ticket based on the following attributes:
- Department: Technical Support, Customer Service, Billing and Payments, Product Support, IT Support, Returns and Exchanges, Sales and Pre-Sales, Human Resources, Service Outages and Maintenance, or General Inquiry
- Priority: low, medium, high
- Type: Incident, Request, Problem, Change
- Language: "en, es, fr, de, etc. (full list at https://docs.aws.amazon.com/translate/latest/dg/what-is-languages.html#what-is-languages-supported)"

Examples:
        [
            {
                "ticket_ID": 1001,
                "ticket_subject": "Discrepancia de facturación en Google Workspace",
                "ticket_body": "Monto de facturación incorrecto.",
                "department": "Billing and Payments",
                "type": "Incident",
                "priority": "low",
                "language": "es"
            },
            {
                "ticket_ID": 1002,
                "ticket_subject": "Urgent Consultation Request for Critical IT Issues",
                "ticket_body": "Critical server issues.",
                "department": "Customer Service",
                "type": "Request",
                "priority": "high",
                "language": "en"
            },
            {
                "ticket_ID": 1003,
                "ticket_subject": "Consulta sobre Servicios de Consultoría en TI",
                "ticket_body": "Info on IT consulting.",
                "department": "General Inquiry",
                "type": "Request",
                "priority": "medium",
                "language": "es"
            },
            {
                "ticket_ID": 1004,
                "ticket_subject": "Demande de mise à jour des dossiers",
                "ticket_body": "Update employee records.",
                "department": "Human Resources",
                "type": "Change",
                "priority": "low",
                "language": "fr"
            },
            {
                "ticket_ID": 1005,
                "ticket_subject": "Issues with Slack connection affecting team communication today",
                "ticket_body": "Problems with Slack.",
                "department": "Product Support",
                "type": "Problem",
                "priority": "medium",
                "language": "en"
            }
        ]
"""

# Define the expected JSON structure for the ticket classification
class TicketClassification(BaseModel):
    ticket_ID: str
    ticket_body: str
    ticket_subject: str
    department: str
    ticket_type: str
    priority: str
    language: str

# Initialize the GenAI client
client = genai.Client(api_key=GOOGLE_API_KEY)

# Load the testing data set
with open('testing_data_set.json', 'r') as file:
    test_json = json.load(file)


final_dict = {}

# Function to classify a ticket
def classify_ticket(ticket: dict) -> dict:
    prompt = f"{system_instructions}\n\nTicket:\nSubject: {ticket['ticket_subject']}\nBody: {ticket['ticket_body']}"

    response = client.models.generate_content(
        model='gemini-2.0-flash',
        contents=prompt,
        config={
            'response_mime_type': 'application/json',
            'response_schema': TicketClassification,
        },
    )
    result = response.parsed
    final_dict[ticket['ticket_ID']] = [result.department, result.ticket_type, result.priority, result.language]

# Classify tickets in the specified range
for i in range(452,500):
    print(f"Classifying ticket {i}")
    classify_ticket(test_json[i])
    time.sleep(3)


Classifying ticket 452


ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource has been exhausted (e.g. check quota).', 'status': 'RESOURCE_EXHAUSTED'}}

In [64]:
final_dict

{}

In [61]:
import csv
# Define the header


header = ['ticket_ID', 'department', 'type', 'priority', 'language']

# Write data to CSV file
with open('submission_genai_1.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    for ticket_id, values in final_dict.items():
        writer.writerow([ticket_id] + values)

print("Data has been written to submission.csv")

Data has been written to submission.csv
